# Bigrams

In [27]:
%run -i "../util/util_simple_classifier.ipynb"
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
(train_df, test_df) = load_train_test_dataset_pd()

Found cached dataset rotten_tomatoes (/home/zhenya/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Found cached dataset rotten_tomatoes (/home/zhenya/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


In [29]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_df=300)
X = bigram_vectorizer.fit_transform(train_df["text"])

In [30]:
print(bigram_vectorizer.get_feature_names_out())
print(len(bigram_vectorizer.get_feature_names_out()))

['10' '10 inch' '10 set' ... 'ótimo esforço' 'últimos' 'últimos tiempos']
40535


In [31]:
first_review = test_df['text'].iat[0]
dense_vector = bigram_vectorizer.transform([first_review]).todense()
print(dense_vector)

[[0 0 0 ... 0 0 0]]


In [32]:
vectorize = lambda x: bigram_vectorizer.transform([x]).toarray()[0]
(X_train, X_test, y_train, y_test) = create_train_test_data(train_df, test_df, vectorize)
clf = train_classifier(X_train, y_train)
test_classifier(test_df, clf)

              precision    recall  f1-score   support

           0       0.75      0.72      0.73       160
           1       0.73      0.76      0.74       160

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320

